In [1]:
import pandas as pd
import numpy as np
import numpy.random as nr
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn.model_selection import GridSearchCV, cross_val_score, StratifiedKFold, train_test_split, KFold
from sklearn.metrics import mean_squared_error, r2_score
import xgboost as xgb
import lightgbm as lgb
import catboost as cat
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam

In [2]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

def sum_missing_values(data):
    cols = list(data.columns)
    
    for col in cols:
        print(f'{col}: {data[col].isna().sum()}')
        
train1 = train.copy()
test1 = test.copy()


for col in list(train1.columns):
    if train1[col].isna().sum() > 17000:
        train1 = train1.drop(col, axis=1)
        
for col in list(test1.columns):
    if test[col].isna().sum() > 3000:
        test1 = test1.drop(col, axis=1)
        
train1 = train1.drop('id', axis=1)
test1 = test1.drop('id', axis=1)

labels = np.array(train['target'])
train1 = train1.drop('target', axis=1)

def fill_mean(data):
    cols = list(data.columns)
    
    for col in cols:
        data[col].fillna(data[col].mean(), inplace=True)
        
    return data

train1['CTR_CATEGO_X'] = train1['CTR_CATEGO_X'].replace('N', 'M')

train1['CTR_CATEGO_X'] = train1['CTR_CATEGO_X'].replace('M', 1)
train1['CTR_CATEGO_X'] = train1['CTR_CATEGO_X'].replace('C', 2)
train1['CTR_CATEGO_X'] = train1['CTR_CATEGO_X'].replace('P', 3)


test1['CTR_CATEGO_X'] = test1['CTR_CATEGO_X'].replace('M', 1)
test1['CTR_CATEGO_X'] = test1['CTR_CATEGO_X'].replace('C', 2)
test1['CTR_CATEGO_X'] = test1['CTR_CATEGO_X'].replace('P', 3)


train2 = train1.copy()
test2 = test1.copy()

#train2 = train2.drop('CTR_CATEGO_X', axis=1)
#test2 = test2.drop('CTR_CATEGO_X', axis=1)

tcrain2 = fill_mean(train2)
test2 = fill_mean(test2)

train_enc = np.array(train2)
test_enc = np.array(test2)

In [3]:
## Randomly splitting into train and test data
nr.seed(9988)
indx = range(train_enc.shape[0])
indx = ms.train_test_split(indx, test_size = 0.2)
x_train = train_enc[indx[0],:]
y_train = np.ravel(labels[indx[0]])
x_test = train_enc[indx[1],:]
y_test = np.ravel(labels[indx[1]])

In [4]:
'''
    Randomly splitting the test data into two parts; one for validation during training
    and the other for testing the model
'''

np.random.seed(9988)
indx1 = range(x_test.shape[0])
indx1 = ms.train_test_split(indx1, test_size = 0.5)
x_test1 = x_test[indx1[0],:]
y_test1 = np.ravel(y_test[indx1[0]])
x_test2 = x_test[indx1[1],:]
y_test2 = np.ravel(y_test[indx1[1]])

In [5]:
cat = cat.CatBoostRegressor(n_estimators=10000,max_depth=12,eval_metric='RMSE')

In [20]:
kf = KFold(n_splits=5,random_state=2020,shuffle=True)

In [21]:
scores = []
test_scores = np.zeros(len(test_enc))
for fold,(tr_in,te_in) in enumerate(kf.split(train2)):
    print(f"================================Fold{fold}====================================")
    X_train,X_test = train2.iloc[tr_in],train2.iloc[te_in]
    y_train,y_test = train['target'].iloc[tr_in],train['target'].iloc[te_in]
    cat.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test,y_test)],early_stopping_rounds=100,use_best_model=True)
    scores.append(np.sqrt(mean_squared_error(y_test,cat.predict(X_test))))
    test_scores += cat.predict(test_enc)
    

================================Fold0====================================
0:	learn: 7.0582313	test: 7.0582313	test1: 6.9891066	best: 6.9891066 (0)	total: 4.21s	remaining: 5h 50m 40s
1:	learn: 7.0272727	test: 7.0272727	test1: 6.9595460	best: 6.9595460 (1)	total: 8.64s	remaining: 5h 59m 59s
2:	learn: 6.9972567	test: 6.9972567	test1: 6.9294041	best: 6.9294041 (2)	total: 12.2s	remaining: 5h 39m 11s
3:	learn: 6.9657398	test: 6.9657398	test1: 6.8984262	best: 6.8984262 (3)	total: 16.2s	remaining: 5h 36m 59s
4:	learn: 6.9373164	test: 6.9373164	test1: 6.8707971	best: 6.8707971 (4)	total: 17.3s	remaining: 4h 47m 15s
5:	learn: 6.9089810	test: 6.9089810	test1: 6.8438060	best: 6.8438060 (5)	total: 19.9s	remaining: 4h 35m 51s
6:	learn: 6.8834129	test: 6.8834129	test1: 6.8194955	best: 6.8194955 (6)	total: 21.7s	remaining: 4h 18m 20s
7:	learn: 6.8557840	test: 6.8557840	test1: 6.7933784	best: 6.7933784 (7)	total: 23.6s	remaining: 4h 5m 21s
8:	learn: 6.8330665	test: 6.8330665	test1: 6.7716222	best: 6.77

74:	learn: 6.2002176	test: 6.2002176	test1: 6.2161132	best: 6.2161132 (74)	total: 2m 16s	remaining: 2h 29m 5s
75:	learn: 6.1963091	test: 6.1963091	test1: 6.2128674	best: 6.2128674 (75)	total: 2m 17s	remaining: 2h 28m 56s
76:	learn: 6.1923672	test: 6.1923672	test1: 6.2101267	best: 6.2101267 (76)	total: 2m 19s	remaining: 2h 28m 46s
77:	learn: 6.1888970	test: 6.1888970	test1: 6.2078925	best: 6.2078925 (77)	total: 2m 21s	remaining: 2h 28m 37s
78:	learn: 6.1850921	test: 6.1850921	test1: 6.2055081	best: 6.2055081 (78)	total: 2m 23s	remaining: 2h 28m 29s
79:	learn: 6.1818179	test: 6.1818179	test1: 6.2036966	best: 6.2036966 (79)	total: 2m 24s	remaining: 2h 28m 23s
80:	learn: 6.1780695	test: 6.1780695	test1: 6.2008053	best: 6.2008053 (80)	total: 2m 26s	remaining: 2h 28m 19s
81:	learn: 6.1737863	test: 6.1737863	test1: 6.1973877	best: 6.1973877 (81)	total: 2m 28s	remaining: 2h 28m 14s
82:	learn: 6.1705334	test: 6.1705334	test1: 6.1956558	best: 6.1956558 (82)	total: 2m 30s	remaining: 2h 28m 8s
83:

148:	learn: 6.0165138	test: 6.0165138	test1: 6.1050275	best: 6.1050275 (148)	total: 4m 28s	remaining: 2h 25m 26s
149:	learn: 6.0151400	test: 6.0151400	test1: 6.1043340	best: 6.1043340 (149)	total: 4m 29s	remaining: 2h 25m 22s
150:	learn: 6.0134837	test: 6.0134837	test1: 6.1037706	best: 6.1037706 (150)	total: 4m 31s	remaining: 2h 25m 18s
151:	learn: 6.0116894	test: 6.0116894	test1: 6.1022914	best: 6.1022914 (151)	total: 4m 33s	remaining: 2h 25m 15s
152:	learn: 6.0100743	test: 6.0100743	test1: 6.1015032	best: 6.1015032 (152)	total: 4m 34s	remaining: 2h 25m 10s
153:	learn: 6.0086508	test: 6.0086508	test1: 6.1006993	best: 6.1006993 (153)	total: 4m 36s	remaining: 2h 25m 7s
154:	learn: 6.0060124	test: 6.0060124	test1: 6.0995898	best: 6.0995898 (154)	total: 4m 38s	remaining: 2h 25m 4s
155:	learn: 6.0040504	test: 6.0040504	test1: 6.0987750	best: 6.0987750 (155)	total: 4m 40s	remaining: 2h 25m
156:	learn: 6.0019693	test: 6.0019693	test1: 6.0979729	best: 6.0979729 (156)	total: 4m 41s	remaining: 

221:	learn: 5.8986321	test: 5.8986321	test1: 6.0556520	best: 6.0556520 (221)	total: 6m 49s	remaining: 2h 26m 50s
222:	learn: 5.8972949	test: 5.8972949	test1: 6.0548777	best: 6.0548777 (222)	total: 6m 51s	remaining: 2h 26m 47s
223:	learn: 5.8959689	test: 5.8959689	test1: 6.0548622	best: 6.0548622 (223)	total: 6m 52s	remaining: 2h 26m 42s
224:	learn: 5.8950129	test: 5.8950129	test1: 6.0544550	best: 6.0544550 (224)	total: 6m 55s	remaining: 2h 27m 5s
225:	learn: 5.8936736	test: 5.8936736	test1: 6.0538783	best: 6.0538783 (225)	total: 6m 59s	remaining: 2h 27m 32s
226:	learn: 5.8927127	test: 5.8927127	test1: 6.0535094	best: 6.0535094 (226)	total: 7m 2s	remaining: 2h 28m 1s
227:	learn: 5.8914837	test: 5.8914837	test1: 6.0529986	best: 6.0529986 (227)	total: 7m 5s	remaining: 2h 28m 31s
228:	learn: 5.8896357	test: 5.8896357	test1: 6.0527082	best: 6.0527082 (228)	total: 7m 10s	remaining: 2h 29m 19s
229:	learn: 5.8885010	test: 5.8885010	test1: 6.0522092	best: 6.0522092 (229)	total: 7m 14s	remaining

294:	learn: 5.8127251	test: 5.8127251	test1: 6.0223699	best: 6.0223699 (294)	total: 11m 34s	remaining: 3h 4m 33s
295:	learn: 5.8112998	test: 5.8112998	test1: 6.0215824	best: 6.0215824 (295)	total: 11m 37s	remaining: 3h 4m 45s
296:	learn: 5.8105086	test: 5.8105086	test1: 6.0209437	best: 6.0209437 (296)	total: 11m 40s	remaining: 3h 4m 57s
297:	learn: 5.8099565	test: 5.8099565	test1: 6.0208656	best: 6.0208656 (297)	total: 11m 44s	remaining: 3h 5m 10s
298:	learn: 5.8091565	test: 5.8091565	test1: 6.0207738	best: 6.0207738 (298)	total: 11m 47s	remaining: 3h 5m 28s
299:	learn: 5.8085792	test: 5.8085792	test1: 6.0205840	best: 6.0205840 (299)	total: 11m 51s	remaining: 3h 5m 45s
300:	learn: 5.8082309	test: 5.8082309	test1: 6.0206500	best: 6.0205840 (299)	total: 11m 54s	remaining: 3h 5m 59s
301:	learn: 5.8070759	test: 5.8070759	test1: 6.0205459	best: 6.0205459 (301)	total: 11m 58s	remaining: 3h 6m 15s
302:	learn: 5.8057377	test: 5.8057377	test1: 6.0199689	best: 6.0199689 (302)	total: 12m 1s	remai

...

440:	learn: 5.6887111	test: 5.6887111	test1: 5.9820937	best: 5.9820937 (440)	total: 19m 33s	remaining: 3h 22m 12s
441:	learn: 5.6873817	test: 5.6873817	test1: 5.9812915	best: 5.9812915 (441)	total: 19m 36s	remaining: 3h 22m 15s
442:	learn: 5.6872366	test: 5.6872366	test1: 5.9812747	best: 5.9812747 (442)	total: 19m 40s	remaining: 3h 22m 18s
443:	learn: 5.6870129	test: 5.6870129	test1: 5.9812203	best: 5.9812203 (443)	total: 19m 43s	remaining: 3h 22m 24s
444:	learn: 5.6869226	test: 5.6869226	test1: 5.9812043	best: 5.9812043 (444)	total: 19m 46s	remaining: 3h 22m 27s
445:	learn: 5.6852663	test: 5.6852663	test1: 5.9806047	best: 5.9806047 (445)	total: 19m 50s	remaining: 3h 22m 34s
446:	learn: 5.6844634	test: 5.6844634	test1: 5.9803531	best: 5.9803531 (446)	total: 19m 53s	remaining: 3h 22m 39s
447:	learn: 5.6843211	test: 5.6843211	test1: 5.9803440	best: 5.9803440 (447)	total: 19m 57s	remaining: 3h 22m 44s
448:	learn: 5.6842089	test: 5.6842089	test1: 5.9803553	best: 5.9803440 (447)	total: 20m	

513:	learn: 5.6162760	test: 5.6162760	test1: 5.9610299	best: 5.9610113 (512)	total: 23m 35s	remaining: 3h 25m 55s
514:	learn: 5.6160480	test: 5.6160480	test1: 5.9609226	best: 5.9609226 (514)	total: 23m 39s	remaining: 3h 25m 58s
515:	learn: 5.6154031	test: 5.6154031	test1: 5.9608250	best: 5.9608250 (515)	total: 23m 42s	remaining: 3h 25m 59s
516:	learn: 5.6126417	test: 5.6126417	test1: 5.9604939	best: 5.9604939 (516)	total: 23m 45s	remaining: 3h 26m 2s
517:	learn: 5.6103787	test: 5.6103787	test1: 5.9600935	best: 5.9600935 (517)	total: 23m 49s	remaining: 3h 26m 6s
518:	learn: 5.6102120	test: 5.6102120	test1: 5.9600901	best: 5.9600901 (518)	total: 23m 52s	remaining: 3h 26m 8s
519:	learn: 5.6083828	test: 5.6083828	test1: 5.9595895	best: 5.9595895 (519)	total: 23m 55s	remaining: 3h 26m 9s
520:	learn: 5.6059618	test: 5.6059618	test1: 5.9592001	best: 5.9592001 (520)	total: 23m 58s	remaining: 3h 26m 9s
521:	learn: 5.6041807	test: 5.6041807	test1: 5.9584943	best: 5.9584943 (521)	total: 24m 2s	re

586:	learn: 5.5491058	test: 5.5491058	test1: 5.9440927	best: 5.9440927 (586)	total: 27m 39s	remaining: 3h 27m 56s
587:	learn: 5.5470349	test: 5.5470349	test1: 5.9437775	best: 5.9437775 (587)	total: 27m 42s	remaining: 3h 27m 57s
588:	learn: 5.5457815	test: 5.5457815	test1: 5.9436142	best: 5.9436142 (588)	total: 27m 46s	remaining: 3h 27m 57s
589:	learn: 5.5444365	test: 5.5444365	test1: 5.9433868	best: 5.9433868 (589)	total: 27m 49s	remaining: 3h 27m 59s
590:	learn: 5.5439102	test: 5.5439102	test1: 5.9432027	best: 5.9432027 (590)	total: 27m 52s	remaining: 3h 28m
591:	learn: 5.5420713	test: 5.5420713	test1: 5.9424679	best: 5.9424679 (591)	total: 27m 56s	remaining: 3h 28m 1s
592:	learn: 5.5409578	test: 5.5409578	test1: 5.9420072	best: 5.9420072 (592)	total: 27m 59s	remaining: 3h 28m 3s
593:	learn: 5.5408081	test: 5.5408081	test1: 5.9419668	best: 5.9419668 (593)	total: 28m 3s	remaining: 3h 28m 4s
594:	learn: 5.5401072	test: 5.5401072	test1: 5.9416559	best: 5.9416559 (594)	total: 28m 10s	rema

659:	learn: 5.4880029	test: 5.4880029	test1: 5.9294992	best: 5.9294992 (659)	total: 31m 56s	remaining: 3h 30m 4s
660:	learn: 5.4878257	test: 5.4878257	test1: 5.9294902	best: 5.9294902 (660)	total: 31m 59s	remaining: 3h 30m 2s
661:	learn: 5.4866554	test: 5.4866554	test1: 5.9291684	best: 5.9291684 (661)	total: 32m 3s	remaining: 3h 30m 1s
662:	learn: 5.4864461	test: 5.4864461	test1: 5.9290812	best: 5.9290812 (662)	total: 32m 6s	remaining: 3h 30m
663:	learn: 5.4861615	test: 5.4861615	test1: 5.9290069	best: 5.9290069 (663)	total: 32m 9s	remaining: 3h 29m 58s
664:	learn: 5.4860637	test: 5.4860637	test1: 5.9289825	best: 5.9289825 (664)	total: 32m 12s	remaining: 3h 29m 59s
665:	learn: 5.4859141	test: 5.4859141	test1: 5.9289711	best: 5.9289711 (665)	total: 32m 16s	remaining: 3h 30m
666:	learn: 5.4838347	test: 5.4838347	test1: 5.9289360	best: 5.9289360 (666)	total: 32m 19s	remaining: 3h 30m 1s
667:	learn: 5.4834141	test: 5.4834141	test1: 5.9287423	best: 5.9287423 (667)	total: 32m 23s	remaining: 

KeyboardInterrupt: 

In [6]:
cat.fit(x_train, y_train, eval_set=[(x_train, y_train), (x_test1, y_test1)], early_stopping_rounds=300)

Learning rate set to 0.019344
0:	learn: 7.0729066	test: 7.0729066	test1: 6.9524502	best: 6.9524502 (0)	total: 1.37s	remaining: 3h 47m 40s
1:	learn: 7.0439612	test: 7.0439612	test1: 6.9247027	best: 6.9247027 (1)	total: 2.52s	remaining: 3h 30m 20s
2:	learn: 7.0175618	test: 7.0175618	test1: 6.8997201	best: 6.8997201 (2)	total: 3.67s	remaining: 3h 23m 47s
3:	learn: 6.9928961	test: 6.9928961	test1: 6.8790325	best: 6.8790325 (3)	total: 4.83s	remaining: 3h 21m 12s
4:	learn: 6.9692215	test: 6.9692215	test1: 6.8574280	best: 6.8574280 (4)	total: 5.98s	remaining: 3h 19m 17s
5:	learn: 6.9427085	test: 6.9427085	test1: 6.8328628	best: 6.8328628 (5)	total: 7.13s	remaining: 3h 18m 1s
6:	learn: 6.9187860	test: 6.9187860	test1: 6.8117349	best: 6.8117349 (6)	total: 8.3s	remaining: 3h 17m 26s
7:	learn: 6.8948430	test: 6.8948430	test1: 6.7924274	best: 6.7924274 (7)	total: 9.46s	remaining: 3h 17m
8:	learn: 6.8713467	test: 6.8713467	test1: 6.7716915	best: 6.7716915 (8)	total: 10.6s	remaining: 3h 16m 38s
9:	l

75:	learn: 6.0631835	test: 6.0631835	test1: 6.1514817	best: 6.1514817 (75)	total: 1m 28s	remaining: 3h 12m 32s
76:	learn: 6.0569427	test: 6.0569427	test1: 6.1482727	best: 6.1482727 (76)	total: 1m 29s	remaining: 3h 12m 28s
77:	learn: 6.0493639	test: 6.0493639	test1: 6.1454819	best: 6.1454819 (77)	total: 1m 30s	remaining: 3h 12m 23s
78:	learn: 6.0429205	test: 6.0429205	test1: 6.1419989	best: 6.1419989 (78)	total: 1m 31s	remaining: 3h 12m 19s
79:	learn: 6.0354930	test: 6.0354930	test1: 6.1388421	best: 6.1388421 (79)	total: 1m 33s	remaining: 3h 12m 16s
80:	learn: 6.0294740	test: 6.0294740	test1: 6.1341451	best: 6.1341451 (80)	total: 1m 34s	remaining: 3h 12m 12s
81:	learn: 6.0230073	test: 6.0230073	test1: 6.1320697	best: 6.1320697 (81)	total: 1m 35s	remaining: 3h 12m 12s
82:	learn: 6.0164487	test: 6.0164487	test1: 6.1294859	best: 6.1294859 (82)	total: 1m 36s	remaining: 3h 12m 9s
83:	learn: 6.0109626	test: 6.0109626	test1: 6.1262449	best: 6.1262449 (83)	total: 1m 37s	remaining: 3h 12m 5s
84:

149:	learn: 5.7267249	test: 5.7267249	test1: 6.0125690	best: 6.0125690 (149)	total: 2m 51s	remaining: 3h 8m 13s
150:	learn: 5.7244549	test: 5.7244549	test1: 6.0123211	best: 6.0123211 (150)	total: 2m 53s	remaining: 3h 8m 13s
151:	learn: 5.7207951	test: 5.7207951	test1: 6.0114229	best: 6.0114229 (151)	total: 2m 54s	remaining: 3h 8m 11s
152:	learn: 5.7161421	test: 5.7161421	test1: 6.0101801	best: 6.0101801 (152)	total: 2m 55s	remaining: 3h 8m 10s
153:	learn: 5.7129169	test: 5.7129169	test1: 6.0092506	best: 6.0092506 (153)	total: 2m 56s	remaining: 3h 8m 9s
154:	learn: 5.7104242	test: 5.7104242	test1: 6.0079451	best: 6.0079451 (154)	total: 2m 57s	remaining: 3h 8m 6s
155:	learn: 5.7067922	test: 5.7067922	test1: 6.0061937	best: 6.0061937 (155)	total: 2m 58s	remaining: 3h 8m 4s
156:	learn: 5.7036653	test: 5.7036653	test1: 6.0048872	best: 6.0048872 (156)	total: 2m 59s	remaining: 3h 8m 2s
157:	learn: 5.7009260	test: 5.7009260	test1: 6.0041482	best: 6.0041482 (157)	total: 3m 1s	remaining: 3h 7m 5

223:	learn: 5.5298737	test: 5.5298737	test1: 5.9552045	best: 5.9552045 (223)	total: 4m 15s	remaining: 3h 6m 6s
224:	learn: 5.5270597	test: 5.5270597	test1: 5.9551450	best: 5.9551450 (224)	total: 4m 16s	remaining: 3h 6m 4s
225:	learn: 5.5245691	test: 5.5245691	test1: 5.9542882	best: 5.9542882 (225)	total: 4m 18s	remaining: 3h 6m 2s
226:	learn: 5.5216313	test: 5.5216313	test1: 5.9543170	best: 5.9542882 (225)	total: 4m 19s	remaining: 3h 6m
227:	learn: 5.5194714	test: 5.5194714	test1: 5.9541118	best: 5.9541118 (227)	total: 4m 20s	remaining: 3h 5m 58s
228:	learn: 5.5192852	test: 5.5192852	test1: 5.9539905	best: 5.9539905 (228)	total: 4m 20s	remaining: 3h 5m 10s
229:	learn: 5.5170128	test: 5.5170128	test1: 5.9533991	best: 5.9533991 (229)	total: 4m 21s	remaining: 3h 5m 8s
230:	learn: 5.5153808	test: 5.5153808	test1: 5.9533897	best: 5.9533897 (230)	total: 4m 22s	remaining: 3h 5m 6s
231:	learn: 5.5136047	test: 5.5136047	test1: 5.9533705	best: 5.9533705 (231)	total: 4m 23s	remaining: 3h 5m 5s
23

297:	learn: 5.3742661	test: 5.3742661	test1: 5.9198599	best: 5.9198599 (297)	total: 5m 38s	remaining: 3h 3m 37s
298:	learn: 5.3722009	test: 5.3722009	test1: 5.9199816	best: 5.9198599 (297)	total: 5m 39s	remaining: 3h 3m 35s
299:	learn: 5.3711128	test: 5.3711128	test1: 5.9198196	best: 5.9198196 (299)	total: 5m 40s	remaining: 3h 3m 34s
300:	learn: 5.3688813	test: 5.3688813	test1: 5.9195155	best: 5.9195155 (300)	total: 5m 41s	remaining: 3h 3m 32s
301:	learn: 5.3678237	test: 5.3678237	test1: 5.9194970	best: 5.9194970 (301)	total: 5m 42s	remaining: 3h 3m 31s
302:	learn: 5.3665638	test: 5.3665638	test1: 5.9193443	best: 5.9193443 (302)	total: 5m 44s	remaining: 3h 3m 29s
303:	learn: 5.3654764	test: 5.3654764	test1: 5.9194466	best: 5.9193443 (302)	total: 5m 45s	remaining: 3h 3m 28s
304:	learn: 5.3643727	test: 5.3643727	test1: 5.9195957	best: 5.9193443 (302)	total: 5m 46s	remaining: 3h 3m 26s
305:	learn: 5.3627236	test: 5.3627236	test1: 5.9187293	best: 5.9187293 (305)	total: 5m 47s	remaining: 3h

...

3868:	learn: 2.3450679	test: 2.3450679	test1: 5.7223296	best: 5.7219089 (3844)	total: 1h 17m 44s	remaining: 2h 3m 14s
3869:	learn: 2.3448393	test: 2.3448393	test1: 5.7222527	best: 5.7219089 (3844)	total: 1h 17m 46s	remaining: 2h 3m 13s
3870:	learn: 2.3443025	test: 2.3443025	test1: 5.7222704	best: 5.7219089 (3844)	total: 1h 17m 47s	remaining: 2h 3m 12s
3871:	learn: 2.3438865	test: 2.3438865	test1: 5.7221888	best: 5.7219089 (3844)	total: 1h 17m 49s	remaining: 2h 3m 11s
3872:	learn: 2.3435286	test: 2.3435286	test1: 5.7222287	best: 5.7219089 (3844)	total: 1h 17m 50s	remaining: 2h 3m 10s
3873:	learn: 2.3433991	test: 2.3433991	test1: 5.7221484	best: 5.7219089 (3844)	total: 1h 17m 52s	remaining: 2h 3m 9s
3874:	learn: 2.3427133	test: 2.3427133	test1: 5.7220524	best: 5.7219089 (3844)	total: 1h 17m 53s	remaining: 2h 3m 9s
3875:	learn: 2.3423877	test: 2.3423877	test1: 5.7219674	best: 5.7219089 (3844)	total: 1h 17m 54s	remaining: 2h 3m 8s
3876:	learn: 2.3417788	test: 2.3417788	test1: 5.7219511	bes

3938:	learn: 2.3156804	test: 2.3156804	test1: 5.7215506	best: 5.7206971 (3907)	total: 1h 19m 20s	remaining: 2h 2m 6s
3939:	learn: 2.3153370	test: 2.3153370	test1: 5.7215404	best: 5.7206971 (3907)	total: 1h 19m 21s	remaining: 2h 2m 5s
3940:	learn: 2.3153324	test: 2.3153324	test1: 5.7215457	best: 5.7206971 (3907)	total: 1h 19m 23s	remaining: 2h 2m 4s
3941:	learn: 2.3152177	test: 2.3152177	test1: 5.7215674	best: 5.7206971 (3907)	total: 1h 19m 24s	remaining: 2h 2m 3s
3942:	learn: 2.3147741	test: 2.3147741	test1: 5.7215862	best: 5.7206971 (3907)	total: 1h 19m 25s	remaining: 2h 2m 2s
3943:	learn: 2.3140454	test: 2.3140454	test1: 5.7217281	best: 5.7206971 (3907)	total: 1h 19m 27s	remaining: 2h 2m 1s
3944:	learn: 2.3132409	test: 2.3132409	test1: 5.7216575	best: 5.7206971 (3907)	total: 1h 19m 28s	remaining: 2h 2m 1s
3945:	learn: 2.3132221	test: 2.3132221	test1: 5.7216576	best: 5.7206971 (3907)	total: 1h 19m 30s	remaining: 2h 2m
3946:	learn: 2.3130848	test: 2.3130848	test1: 5.7215534	best: 5.720

4008:	learn: 2.2880686	test: 2.2880686	test1: 5.7221992	best: 5.7206971 (3907)	total: 1h 20m 55s	remaining: 2h 57s
4009:	learn: 2.2877823	test: 2.2877823	test1: 5.7223208	best: 5.7206971 (3907)	total: 1h 20m 56s	remaining: 2h 56s
4010:	learn: 2.2873952	test: 2.2873952	test1: 5.7222981	best: 5.7206971 (3907)	total: 1h 20m 58s	remaining: 2h 55s
4011:	learn: 2.2873507	test: 2.2873507	test1: 5.7222949	best: 5.7206971 (3907)	total: 1h 20m 58s	remaining: 2h 53s
4012:	learn: 2.2867640	test: 2.2867640	test1: 5.7223916	best: 5.7206971 (3907)	total: 1h 21m	remaining: 2h 52s
4013:	learn: 2.2863179	test: 2.2863179	test1: 5.7223464	best: 5.7206971 (3907)	total: 1h 21m 1s	remaining: 2h 51s
4014:	learn: 2.2857887	test: 2.2857887	test1: 5.7224234	best: 5.7206971 (3907)	total: 1h 21m 2s	remaining: 2h 50s
4015:	learn: 2.2857851	test: 2.2857851	test1: 5.7224298	best: 5.7206971 (3907)	total: 1h 21m 4s	remaining: 2h 49s
4016:	learn: 2.2857090	test: 2.2857090	test1: 5.7224394	best: 5.7206971 (3907)	total: 1

4080:	learn: 2.2611787	test: 2.2611787	test1: 5.7225719	best: 5.7206971 (3907)	total: 1h 22m 33s	remaining: 1h 59m 45s
4081:	learn: 2.2605365	test: 2.2605365	test1: 5.7225377	best: 5.7206971 (3907)	total: 1h 22m 34s	remaining: 1h 59m 44s
4082:	learn: 2.2601064	test: 2.2601064	test1: 5.7225479	best: 5.7206971 (3907)	total: 1h 22m 35s	remaining: 1h 59m 43s
4083:	learn: 2.2596253	test: 2.2596253	test1: 5.7226622	best: 5.7206971 (3907)	total: 1h 22m 37s	remaining: 1h 59m 42s
4084:	learn: 2.2593076	test: 2.2593076	test1: 5.7226075	best: 5.7206971 (3907)	total: 1h 22m 38s	remaining: 1h 59m 41s
4085:	learn: 2.2591787	test: 2.2591787	test1: 5.7226401	best: 5.7206971 (3907)	total: 1h 22m 40s	remaining: 1h 59m 40s
4086:	learn: 2.2587877	test: 2.2587877	test1: 5.7227574	best: 5.7206971 (3907)	total: 1h 22m 41s	remaining: 1h 59m 39s
4087:	learn: 2.2582588	test: 2.2582588	test1: 5.7228769	best: 5.7206971 (3907)	total: 1h 22m 42s	remaining: 1h 59m 38s
4088:	learn: 2.2581837	test: 2.2581837	test1: 5.

4150:	learn: 2.2346649	test: 2.2346649	test1: 5.7224273	best: 5.7206971 (3907)	total: 1h 24m 7s	remaining: 1h 58m 33s
4151:	learn: 2.2342940	test: 2.2342940	test1: 5.7224982	best: 5.7206971 (3907)	total: 1h 24m 8s	remaining: 1h 58m 32s
4152:	learn: 2.2342743	test: 2.2342743	test1: 5.7225123	best: 5.7206971 (3907)	total: 1h 24m 8s	remaining: 1h 58m 29s
4153:	learn: 2.2341892	test: 2.2341892	test1: 5.7225244	best: 5.7206971 (3907)	total: 1h 24m 10s	remaining: 1h 58m 28s
4154:	learn: 2.2338430	test: 2.2338430	test1: 5.7226071	best: 5.7206971 (3907)	total: 1h 24m 11s	remaining: 1h 58m 27s
4155:	learn: 2.2333820	test: 2.2333820	test1: 5.7225895	best: 5.7206971 (3907)	total: 1h 24m 12s	remaining: 1h 58m 26s
4156:	learn: 2.2330175	test: 2.2330175	test1: 5.7226002	best: 5.7206971 (3907)	total: 1h 24m 14s	remaining: 1h 58m 25s
4157:	learn: 2.2330066	test: 2.2330066	test1: 5.7226159	best: 5.7206971 (3907)	total: 1h 24m 15s	remaining: 1h 58m 24s
4158:	learn: 2.2324569	test: 2.2324569	test1: 5.722

In [7]:
#making prediction on the other part of the test data

pred3 = cat.predict(x_test2)
mean_squared_error(pred3, y_test2) ** 0.5

5.5286695802314485

In [8]:
#preparing the predictions make into the sample file for submission

preds1 = cat.predict(test_enc)
sample1 = pd.read_csv('SampleSubmission.csv')
test = pd.read_csv('test.csv')
sample1.id = test.id
sample1.target = preds1
sample1.to_csv('CAT.csv', index = False)